In [ ]:
import requests
import sqlite3

# Function to fetch PrizePicks props
def fetch_prizepicks_data():
    """
    Fetch PrizePicks props using OddsAPI or mock equivalent.
    :return: Normalized list of PrizePicks player props.
    """
    odds_api_key = "your_oddsapi_key"  # Replace with your actual OddsAPI key
    url = f"https://api.the-odds-api.com/v4/sports/basketball_nba/odds/?regions=us&markets=player_props&apiKey={odds_api_key}"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching PrizePicks data: {e}")
        return []

    prizepicks_props = []

    # Normalize data for comparison
    for event in data:
        for bookmaker in event.get("bookmakers", []):
            if bookmaker["key"] == "prizepicks":  # Filter for PrizePicks data
                for market in bookmaker.get("markets", []):
                    player = market.get("outcome", {}).get("name", "")
                    stat = market.get("key", "")
                    line = market.get("outcome", {}).get("point", 0)

                    # Append normalized prop
                    prizepicks_props.append({
                        "event": event["id"],
                        "player": player,
                        "stat": stat,
                        "line": line
                    })

    return prizepicks_props

# Function to store PrizePicks data in the database
def store_prizepicks_data(props):
    """
    Store PrizePicks props in the database.
    :param props: List of PrizePicks player props.
    """
    connection = sqlite3.connect("odds_db.db")
    cursor = connection.cursor()

    # Add a table for PrizePicks props if not already present
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS PrizePicksProps (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        event TEXT,
        player TEXT,
        stat TEXT,
        line REAL
    )
    """)

    # Insert props into the table
    for prop in props:
        cursor.execute("""
        INSERT INTO PrizePicksProps (event, player, stat, line)
        VALUES (?, ?, ?, ?)
        """, (prop["event"], prop["player"], prop["stat"], prop["line"]))

    connection.commit()
    connection.close()

if __name__ == "__main__":
    # Fetch PrizePicks data
    prizepicks_props = fetch_prizepicks_data()
    print("Fetched PrizePicks Props:")
    for prop in prizepicks_props:
        print(prop)

    # Store PrizePicks data
    if prizepicks_props:
        store_prizepicks_data(prizepicks_props)
        print("Stored PrizePicks props in the database.")


In [ ]:
import sqlite3

# Function to compare PrizePicks props with sportsbook odds
def compare_prizepicks_with_sportsbooks():
    """
    Compare PrizePicks player props with sportsbook odds to find discrepancies.
    :return: List of discrepancies.
    """
    # Step 1: Connect to the SQLite database
    connection = sqlite3.connect("odds_db.db")
    cursor = connection.cursor()

    # Step 2: Fetch all PrizePicks props from the PrizePicksProps table
    cursor.execute("SELECT event, player, stat, line FROM PrizePicksProps")
    prizepicks_data = cursor.fetchall()  # Returns a list of tuples [(event, player, stat, line), ...]

    # Step 3: Fetch all sportsbook odds from the OddsTable
    cursor.execute("SELECT event, team, bookmaker, odds FROM OddsTable")
    sportsbook_data = cursor.fetchall()  # Returns a list of tuples [(event, team, bookmaker, odds), ...]

    # Step 4: Close the database connection after fetching the data
    connection.close()

    # Step 5: Initialize an empty list to store discrepancies
    discrepancies = []

    # Step 6: Compare each PrizePicks prop with sportsbook odds
    for pp_event, pp_player, pp_stat, pp_line in prizepicks_data:
        for sb_event, sb_team, sb_bookmaker, sb_odds in sportsbook_data:
            # Step 6.1: Match PrizePicks and sportsbook data by event, player/team, and stat type
            if pp_event == sb_event and pp_player == sb_team and pp_stat == "points":  # Example comparison for "points"

                # Step 6.2: Calculate implied probabilities for sportsbook and PrizePicks odds
                # Sportsbook implied probability
                sb_implied_prob = 1 / sb_odds if sb_odds > 0 else abs(sb_odds) / (100 + abs(sb_odds))

                # PrizePicks implied probability (assume it's based on the line for simplicity here)
                pp_implied_prob = 1 / pp_line if pp_line > 0 else abs(pp_line) / (100 + abs(pp_line))

                # Step 6.3: Calculate the absolute difference between implied probabilities
                discrepancy = abs(pp_implied_prob - sb_implied_prob)

                # Step 6.4: Check if the discrepancy exceeds a threshold (e.g., 0.05 or 5%)
                if discrepancy > 0.05:  # Significant discrepancy

                    # Step 6.5: Append the discrepancy to the list
                    discrepancies.append({
                        "event": pp_event,          # Event name or ID
                        "player": pp_player,        # Player name
                        "stat": pp_stat,            # Statistic type (e.g., points)
                        "prizepicks_line": pp_line, # Line value from PrizePicks
                        "bookmaker": sb_bookmaker,  # Bookmaker offering the odds
                        "bookmaker_odds": sb_odds,  # Odds from the bookmaker
                        "discrepancy": discrepancy  # Calculated discrepancy
                    })

    # Step 7: Return the list of discrepancies
    return discrepancies

def save_discrepancies_to_db(discrepancies):
    """
    Save the discrepancies to the database for further analysis or visualization.
    :param discrepancies: List of discrepancy dictionaries.
    """
    # Step 1: Connect to the SQLite database
    connection = sqlite3.connect("odds_db.db")
    cursor = connection.cursor()

    # Step 2: Create a table for storing discrepancies if it doesn't already exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Discrepancies (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        event TEXT,
        player TEXT,
        stat TEXT,
        prizepicks_line REAL,
        bookmaker TEXT,
        bookmaker_odds REAL,
        discrepancy REAL,
        ev REAL
    )
    """)

    # Step 3: Insert each discrepancy into the table
    for d in discrepancies:
        cursor.execute("""
        INSERT INTO Discrepancies (event, player, stat, prizepicks_line, bookmaker, bookmaker_odds, discrepancy, ev)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (d["event"], d["player"], d["stat"], d["prizepicks_line"], d["bookmaker"], d["bookmaker_odds"], d["discrepancy"], d["ev"]))

    # Step 4: Commit the transaction and close the connection
    connection.commit()
    connection.close()

def highlight_discrepancies(discrepancies, prizepicks_payout=1.25):
    """
    Highlight discrepancies and calculate expected value (EV) for PrizePicks props.
    :param discrepancies: List of discrepancy dictionaries.
    :param prizepicks_payout: Payout multiplier for PrizePicks bets (e.g., 1.25x).
    :return: List of discrepancies with EV calculations.
    """
    highlighted = []

    print("Highlighting Discrepancies with EV Calculation:")
    for d in discrepancies:
        # Calculate fair odds from the bookmaker's implied probability
        sb_implied_prob = 1 / d["bookmaker_odds"] if d["bookmaker_odds"] > 0 else abs(d["bookmaker_odds"]) / (100 + abs(d["bookmaker_odds"]))

        # Calculate EV using PrizePicks payout and sportsbook fair odds
        ev = (sb_implied_prob * prizepicks_payout) - (1 - sb_implied_prob)

        # Add EV to the discrepancy dictionary
        d["ev"] = ev
        highlighted.append(d)

        # Print the result in a readable format
        print(
            f"Event: {d['event']}, Player: {d['player']}, Stat: {d['stat']}\n"
            f"PrizePicks Line: {d['prizepicks_line']}, Bookmaker: {d['bookmaker']}, "
            f"Bookmaker Odds: {d['bookmaker_odds']}, Discrepancy: {d['discrepancy']:.2f}, EV: {ev:.2f}\n"
        )

    return highlighted

if __name__ == "__main__":
    # Step 1: Compare PrizePicks props with sportsbook odds
    discrepancies = compare_prizepicks_with_sportsbooks()

    # Step 2: Highlight discrepancies and calculate EV
    highlighted_discrepancies = highlight_discrepancies(discrepancies)

    # Step 3: Save highlighted discrepancies with EV to the database
    save_discrepancies_to_db(highlighted_discrepancies)

    # Step 4: Print a summary for testing purposes
    print("\nTest Results:")
    for discrepancy in highlighted_discrepancies:
        assert discrepancy["discrepancy"] > 0.05, "Discrepancy threshold check failed"
        assert discrepancy["ev"] is not None, "EV calculation missing"
        print(discrepancy)

    print("All tests passed successfully!")

